In [1]:
%matplotlib inline
import numpy as np
import pylab as plt
from matplotlib.gridspec import GridSpec
from astropy.table import Table, vstack, Column, MaskedColumn
from astropy import units as u
from scipy.stats import kde
from scipy.stats import binned_statistic as b_s
from scipy.stats import binned_statistic_2d as bs2d
from astropy import units as un
from astropy import constants as con
from matplotlib.lines import Line2D

from astropy.cosmology import Planck15
from tqdm import trange
from scipy.stats import ks_2samp
from scipy import special

import os
os.environ['PATH'] = os.environ['PATH'] + ':/usr/texbin'

plt.rc('path', simplify=True)
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('axes', labelsize='xx-large', facecolor='none', linewidth=0.7)
plt.rc('xtick', labelsize='x-large')
plt.rc('ytick', labelsize='x-large')
plt.rc('lines', markersize=4, linewidth=1, markeredgewidth=0.2)
plt.rc('legend', numpoints=1, frameon=False, handletextpad=0.3, scatterpoints=1, handlelength=2, handleheight=0.1)
#plt.rc('savefig', facecolor='none', frameon='False')

params =   {'font.size' : 16,
            'xtick.major.size': 8,
            'ytick.major.size': 8,
            'xtick.minor.size': 3,
            'ytick.minor.size': 3,
            }
plt.rcParams.update(params) 

np.warnings.filterwarnings('ignore')

def legend_outside_plot(ax,location='top',ncol=3,pad=0, handles=None, labels=None):
    if handles==None:
        handles, labels = ax.get_legend_handles_labels()
    if location=='top':
        leg = ax.legend(handles=handles, labels=labels, bbox_to_anchor=(0,1.02+pad,1,0.2), loc="lower left",
                mode="expand", borderaxespad=0, ncol=ncol)
    else:
        print("Location",location,"not implemented. Please code up")
    return leg

In [2]:
data = Table.read('data.txt', format = 'ascii' )

In [3]:
data.rename_column('col1', 'RA')  
data.rename_column('col2', 'Dec') 
data.rename_column('col3', 'Imag') 
data.rename_column('col4', 'S')  
data.rename_column('col5', 'zspec') 
data.rename_column('col6', 'N')  
data.rename_column('col7', 'zG')  
data.rename_column('col8', 'chi2G')  
data.rename_column('col9', 'LEll')  
data.rename_column('col10', 'LSbc') 
data.rename_column('col11', 'LIrr')
data.rename_column('col12', 'zGA') 
data.rename_column('col13', 'chi2GA') 
data.rename_column('col14', 'LAGNp') 
data.rename_column('col15', 'LELLp') 
data.rename_column('col16', 'LSbcp') 
data.rename_column('col17', 'LIrrp') 
data.rename_column('col18', 'F') 
data.rename_column('col19', 'EBV') 
data.rename_column('col20', 'Chi2S') 
data.rename_column('col21', 'SType') 

In [4]:
l0 = data[data['LAGNp']!=0.0]

In [5]:
l0

RA,Dec,Imag,S,zspec,N,zG,chi2G,LEll,LSbc,LIrr,zGA,chi2GA,LAGNp,LELLp,LSbcp,LIrrp,F,EBV,Chi2S,SType
float64,float64,float64,float64,float64,str4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str5
217.18223,32.30523,22.77,0.95,-99.99,8/3,1.27,2.53,0.0,0.829,0.361,1.2,2.9,0.651,0.0,0.768,0.0,0.54,0.1,8.92,1500K
217.86705,32.30527,20.02,0.03,-99.99,12/0,0.54,37.08,0.44,0.47,-0.461,0.51,45.75,0.268,0.546,0.0,-0.237,0.24,0.5,56.73,k/m
217.88212,32.30531,22.67,0.03,-99.99,11/1,0.85,0.55,0.0,0.0,0.121,0.81,0.63,-0.31,0.0,0.0,-0.052,0.53,0.04,1.09,f/g
217.84705,32.30564,20.44,0.03,-99.99,11/0,1.08,18.79,0.0,1.319,1.2,1.01,23.96,0.802,0.0,1.278,0.947,0.24,0.05,88.09,f/g
217.70446,32.30566,19.34,0.92,-99.99,11/0,0.19,117.86,-0.28,-0.986,0.0,0.15,129.12,-0.636,-0.513,0.0,0.0,0.74,1.0,82.43,k/m
217.48821,32.30604,19.71,0.03,-99.99,9/1,0.41,1.13,0.0,-1.56,0.629,0.37,0.63,-0.693,0.0,0.0,0.51,2.96,0.0,29.92,f/g
217.36268,32.3061,18.71,0.03,-99.99,12/0,0.21,12.43,0.0,-0.735,0.325,0.21,13.6,-0.711,0.0,0.0,0.326,0.7,0.0,57.12,f/g
217.24316,32.30626,22.34,0.95,-99.99,9/2,1.08,2.14,0.0,-1.455,0.627,0.83,1.56,0.239,0.0,-0.597,0.013,2.28,0.03,3.9,f/g
217.16662,32.30637,21.26,0.13,-99.99,10/1,0.69,3.23,0.0,0.0,0.531,0.62,3.48,-0.314,-1.659,0.0,0.358,0.78,0.0,5.63,f/g


In [6]:
LAGNerg = MaskedColumn(name = "LAGNerg", data = ((l0['LAGNp'])+43.58) , dtype=np.float64) # 44 or 43?
l0.add_column(LAGNerg)
#l0.replace_column('LAGNerg',LAGNerg)



logF = MaskedColumn(name = "logF", data = np.log10(l0['F']) , dtype=np.float64) 
l0.add_column(logF)
#l0.replace_column('logF',logF)

In [7]:
I1 = np.array(l0['Imag'])
I = sorted(I1, reverse=False)
I = MaskedColumn(name = "I", data = sorted(I1, reverse=False)  , dtype=np.float64)
l0.add_column(I)
#l0.replace_column('I',I )
print(len(I))


#adding indexing
index = np.linspace(1,305293,305293)
index = MaskedColumn(name = "index", data = np.linspace(1,305293,305293)  , dtype=np.float64)
l0.add_column(index)
#l0.replace_column('index',index )


#for SA
SA = MaskedColumn(name = "SA", data = [i/8.54 for i in l0['index']]  , dtype=np.float64)
l0.add_column(SA)
#l0.replace_column('SA',SA )

#splitting by F-ratio as in Figure 17

F10 = l0[l0['F']>=10]
I1 = np.array(F10['Imag'])
I10 = sorted(I1, reverse=False)
L1 = np.array(F10['LAGNerg'])
L10 = sorted(L1, reverse=False)
SA10 = [i/8.54 for i in F10['index']]
F5 = l0[l0['F']>=5]
I1 = np.array(F5['Imag'])
I5 = sorted(I1, reverse=False)
L1 = np.array(F5['LAGNerg'])
L5 = sorted(L1, reverse=False)
SA5 = [i/8.54 for i in F5['index']]
F20 = l0[l0['F']>=20]
I1 = np.array(F20['Imag'])
I20 = sorted(I1, reverse=False)
L1 = np.array(F20['LAGNerg'])
L20 = sorted(L1, reverse=False)
SA20 = [i/8.54 for i in F20['index']]

print (len(L10))
print (len(L5))
print (len(L20))



305293
16907
41476
5760


In [8]:
F=8
FM = data[data['F']>=F]
FL = data[data['F']<=F]
print (len(FM))
print (len(FL))

23225
407850


In [9]:
p = 0.01
prob = 1-(p/(1-2*p))*((len(FL)/len(FM))-1)
print (prob)
print (len(FM)/len(FL))

0.8310120603677423
0.0569449552531568


In [10]:
I1 = np.array(l0['Imag'])
I = sorted(I1, reverse=False)
I = MaskedColumn(name = "I", data = sorted(I1, reverse=False)  , dtype=np.float64)
#l0.add_column(I)
l0.replace_column('I',I )
print(len(I))


#adding indexing
index = np.linspace(1,305293,305293)
index = MaskedColumn(name = "index", data = np.linspace(1,305293,305293)  , dtype=np.float64)
#l0.add_column(index)
l0.replace_column('index',index )


#for SA
SA = MaskedColumn(name = "SA", data = [i/8.54 for i in l0['index']]  , dtype=np.float64)
#l0.add_column(SA)
l0.replace_column('SA',SA )

#splitting by F-ratio as in Figure 17

F10 = l0[l0['F']>=10]
I1 = np.array(F10['Imag'])
I10 = sorted(I1, reverse=False)
L1 = np.array(F10['LAGNerg'])
L10 = sorted(L1, reverse=False)
SA10 = [i/8.54 for i in F10['index']]
F5 = l0[l0['F']>=5]
I1 = np.array(F5['Imag'])
I5 = sorted(I1, reverse=False)
L1 = np.array(F5['LAGNerg'])
L5 = sorted(L1, reverse=False)
SA5 = [i/8.54 for i in F5['index']]
F20 = l0[l0['F']>=20]
I1 = np.array(F20['Imag'])
I20 = sorted(I1, reverse=False)
L1 = np.array(F20['LAGNerg'])
L20 = sorted(L1, reverse=False)
SA20 = [i/8.54 for i in F20['index']]

print (len(L10))
print (len(L5))
print (len(L20))



305293
16907
41476
5760


In [11]:
#need to sum the galaxy luminosity so change to log ergs then unlog and add then re log
#the change from l0 to lg0 is removing the sample that have l = 0 from lg as well 
#check the numbers in sample for lg0 with the numbers for l0 as this mught affect size of cut? though maybe ignore lg0?
#note by limiting samplee not only by non zero Lagn but also by non zero Lg's we are reducing the number of data points, potentially significantly
#lets check by how much..
# cuts it down considerably I wonder if it would be better to keep if not all of them are 0 just not add the zero  though is that not what i did before?
#careful with when i re-run this box because the orignal won't run without taking the 3's back out 
#find and replace all the 0's with a number far larger than real then after adding make a range cut to take them out ?


LSbcerg = MaskedColumn(name = "LSbcerg", data = ((l0['LSbc'])+43.58) , dtype=np.float64) # 44 or 43?
l0.add_column(LSbcerg)
#l0.replace_column('LSbcerg',LSbcerg)

LEllerg = MaskedColumn(name = "LEllerg", data = ((l0['LEll'])+43.58) , dtype=np.float64) # 44 or 43?
l0.add_column(LEllerg)
#l0.replace_column('LEllerg',LEllerg)

LIrrerg = MaskedColumn(name = "LIrrerg", data = ((l0['LIrr'])+43.58) , dtype=np.float64) # 44 or 43?
l0.add_column(LIrrerg)
#l0.replace_column('LIrrerg',LIrrerg)


Lg = MaskedColumn(name = "Lg", data = list(map(sum, zip(10**(l0['LEllerg']),10**(l0['LSbcerg']),10**(l0['LIrrerg']))))  , dtype=np.float64)
l0.add_column(Lg)
#l0.replace_column('Lg',Lg )

LGlog =  MaskedColumn(name = "LGlog", data = np.log10(l0['Lg']) , dtype=np.float64) 
l0.add_column(LGlog)
#l0.replace_column('LGlog',LGlog)


F10 = l0[l0['F']>=10]
I1 = np.array(F10['Imag'])
I10 = sorted(I1, reverse=False)
L1 = np.array(F10['LAGNerg'])
L10 = sorted(L1, reverse=False)
SA10 = [i/8.54 for i in F10['index']]
F5 = l0[l0['F']>=5]
I1 = np.array(F5['Imag'])
I5 = sorted(I1, reverse=False)
L1 = np.array(F5['LAGNerg'])
L5 = sorted(L1, reverse=False)
SA5 = [i/8.54 for i in F5['index']]
F20 = l0[l0['F']>=20]
I1 = np.array(F20['Imag'])
I20 = sorted(I1, reverse=False)
L1 = np.array(F20['LAGNerg'])
L20 = sorted(L1, reverse=False)
SA20 = [i/8.54 for i in F20['index']]

print (len(l0['LAGNerg']))
print (len(F5['LAGNerg']))
print (len(F10['LAGNerg']))
print (len(F20['LAGNerg']))

305293
41476
16907
5760


In [12]:
l0

RA,Dec,Imag,S,zspec,N,zG,chi2G,LEll,LSbc,LIrr,zGA,chi2GA,LAGNp,LELLp,LSbcp,LIrrp,F,EBV,Chi2S,SType,LAGNerg,logF,I,index,SA,LSbcerg,LEllerg,LIrrerg,Lg,LGlog
float64,float64,float64,float64,float64,str4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str5,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
217.18223,32.30523,22.77,0.95,-99.99,8/3,1.27,2.53,0.0,0.829,0.361,1.2,2.9,0.651,0.0,0.768,0.0,0.54,0.1,8.92,1500K,44.231,-0.26760624017703144,-30.18,1.0,0.117096018735363,44.409,43.58,43.940999999999995,3.817644801226373e+44,44.58179551859838
217.86705,32.30527,20.02,0.03,-99.99,12/0,0.54,37.08,0.44,0.47,-0.461,0.51,45.75,0.268,0.546,0.0,-0.237,0.24,0.5,56.73,k/m,43.848,-0.619788758288394,-29.15,2.0,0.234192037470726,44.05,44.019999999999996,43.119,2.3006694855720697e+44,44.36185423235735
217.88212,32.30531,22.67,0.03,-99.99,11/1,0.85,0.55,0.0,0.0,0.121,0.81,0.63,-0.31,0.0,0.0,-0.052,0.53,0.04,1.09,f/g,43.269999999999996,-0.27572413039921095,-28.79,3.0,0.35128805620608905,43.58,43.58,43.701,1.2627213821635071e+44,44.10130753461119
217.84705,32.30564,20.44,0.03,-99.99,11/0,1.08,18.79,0.0,1.319,1.2,1.01,23.96,0.802,0.0,1.278,0.947,0.24,0.05,88.09,f/g,44.382,-0.619788758288394,-28.42,4.0,0.468384074941452,44.899,43.58,44.78,1.4330798561868888e+45,45.15627039146854
217.70446,32.30566,19.34,0.92,-99.99,11/0,0.19,117.86,-0.28,-0.986,0.0,0.15,129.12,-0.636,-0.513,0.0,0.0,0.74,1.0,82.43,k/m,42.943999999999996,-0.13076828026902382,-28.27,5.0,0.5854800936768151,42.594,43.3,43.58,6.189801213574065e+43,43.791676701809436
217.48821,32.30604,19.71,0.03,-99.99,9/1,0.41,1.13,0.0,-1.56,0.629,0.37,0.63,-0.693,0.0,0.0,0.51,2.96,0.0,29.92,f/g,42.887,0.4712917110589386,-28.0,6.0,0.7025761124121781,42.019999999999996,43.58,44.208999999999996,2.00874071944412e+44,44.302923883297964
217.36268,32.3061,18.71,0.03,-99.99,12/0,0.21,12.43,0.0,-0.735,0.325,0.21,13.6,-0.711,0.0,0.0,0.326,0.7,0.0,57.12,f/g,42.869,-0.1549019599857432,-27.95,7.0,0.8196721311475411,42.845,43.58,43.905,1.2536997181064061e+44,44.098193528212455
217.24316,32.30626,22.34,0.95,-99.99,9/2,1.08,2.14,0.0,-1.455,0.627,0.83,1.56,0.239,0.0,-0.597,0.013,2.28,0.03,3.9,f/g,43.818999999999996,0.35793484700045375,-27.94,8.0,0.936768149882904,42.125,43.58,44.207,2.004170245820466e+44,44.3019346102491
217.16662,32.30637,21.26,0.13,-99.99,10/1,0.69,3.23,0.0,0.0,0.531,0.62,3.48,-0.314,-1.659,0.0,0.358,0.78,0.0,5.63,f/g,43.266,-0.10790539730951958,-27.79,9.0,1.053864168618267,43.58,43.58,44.111,2.051598066254645e+44,44.31209228104719


In [13]:
#add columns unlogging 
LSbc1 = MaskedColumn(name = "LSbc1", data = (10**(l0['LSbcerg'])) , dtype=np.float64) 
l0.add_column(LSbc1)
#l0.replace_column('LSbc1',LSbc1)

LEll1 = MaskedColumn(name = "LEll1", data = (10**(l0['LEllerg'])) , dtype=np.float64) 
l0.add_column(LEll1)
#l0.replace_column('LEll1',LEll1)

LIrr1 = MaskedColumn(name = "LIrr1", data = (10**(l0['LIrrerg'])) , dtype=np.float64) 
l0.add_column(LIrr1)
#l0.replace_column('LIrr1',LIrr1)



#thennn add columns replacing 43.58 with 0
LSbc2 = MaskedColumn(name = "LSbc2", data = (np.where(l0['LSbc1']==10**43.58,0, l0['LSbc1'])) , dtype=np.float64) 
l0.add_column(LSbc2)
#l0.replace_column('LSbc2',LSbc2)

LEll2 = MaskedColumn(name = "LEll2", data = (np.where(l0['LEll1']==10**43.58,0, l0['LEll1'])) , dtype=np.float64) 
l0.add_column(LEll2)
#l0.replace_column('LEll2',LEll2)

LIrr2 = MaskedColumn(name = "LIrr2", data = (np.where(l0['LIrr1']==10**43.58,0, l0['LIrr1'])) , dtype=np.float64) 
l0.add_column(LIrr2)
#l0.replace_column('LIrr2',LIrr2)

#then add lg column (sum them)
Lg = MaskedColumn(name = "Lg", data = list(map(sum, zip(l0['LEll2'],l0['LSbc2'],l0['LIrr2'])))  , dtype=np.float64)
#l0.add_column(Lg)
l0.replace_column('Lg',Lg )



#then add log column 
LGlog =  MaskedColumn(name = "LGlog", data = np.log10(l0['Lg']) , dtype=np.float64) 
#l0.add_column(LGlog)
l0.replace_column('LGlog',LGlog)

#remove the values where they are all 0
lg0 = l0[[np.isfinite(l0['LGlog'])]]

F10 = lg0[lg0['F']>=10]
I1 = np.array(F10['Imag'])
I10 = sorted(I1, reverse=False)
L1 = np.array(F10['LAGNerg'])
L10 = sorted(L1, reverse=False)
SA10 = [i/8.54 for i in F10['index']]
F5 = lg0[lg0['F']>=5]
I1 = np.array(F5['Imag'])
I5 = sorted(I1, reverse=False)
L1 = np.array(F5['LAGNerg'])
L5 = sorted(L1, reverse=False)
SA5 = [i/8.54 for i in F5['index']]
F20 = lg0[lg0['F']>=20]
I1 = np.array(F20['Imag'])
I203 = sorted(I1, reverse=False)
L1 = np.array(F20['LAGNerg'])
L20 = sorted(L1, reverse=False)
SA20 = [i/8.54 for i in F20['index']]

print (len(lg0['LAGNerg']))
print (len(F5['LAGNerg']))
print (len(F10['LAGNerg']))
print (len(F20['LAGNerg']))

305254
41470
16907
5760


In [14]:
lg0

RA,Dec,Imag,S,zspec,N,zG,chi2G,LEll,LSbc,LIrr,zGA,chi2GA,LAGNp,LELLp,LSbcp,LIrrp,F,EBV,Chi2S,SType,LAGNerg,logF,I,index,SA,LSbcerg,LEllerg,LIrrerg,Lg,LGlog,LSbc1,LEll1,LIrr1,LSbc2,LEll2,LIrr2
float64,float64,float64,float64,float64,str4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str5,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
217.18223,32.30523,22.77,0.95,-99.99,8/3,1.27,2.53,0.0,0.829,0.361,1.2,2.9,0.651,0.0,0.768,0.0,0.54,0.1,8.92,1500K,44.231,-0.26760624017703144,-30.18,1.0,0.117096018735363,44.409,43.58,43.940999999999995,3.437455404905813e+44,44.53623707264808,2.5644840365177108e+44,3.801893963205597e+43,8.729713683881023e+43,2.5644840365177108e+44,0.0,8.729713683881023e+43
217.86705,32.30527,20.02,0.03,-99.99,12/0,0.54,37.08,0.44,0.47,-0.461,0.51,45.75,0.268,0.546,0.0,-0.237,0.24,0.5,56.73,k/m,43.848,-0.619788758288394,-29.15,2.0,0.234192037470726,44.05,44.019999999999996,43.119,2.3006694855720697e+44,44.36185423235735,1.1220184543019561e+44,1.0471285480508899e+44,1.3152248321922376e+43,1.1220184543019561e+44,1.0471285480508899e+44,1.3152248321922376e+43
217.88212,32.30531,22.67,0.03,-99.99,11/1,0.85,0.55,0.0,0.0,0.121,0.81,0.63,-0.31,0.0,0.0,-0.052,0.53,0.04,1.09,f/g,43.269999999999996,-0.27572413039921095,-28.79,3.0,0.35128805620608905,43.58,43.58,43.701,5.023425895223877e+43,43.701,3.801893963205597e+43,3.801893963205597e+43,5.023425895223877e+43,0.0,0.0,5.023425895223877e+43
217.84705,32.30564,20.44,0.03,-99.99,11/0,1.08,18.79,0.0,1.319,1.2,1.01,23.96,0.802,0.0,1.278,0.947,0.24,0.05,88.09,f/g,44.382,-0.619788758288394,-28.42,4.0,0.468384074941452,44.899,43.58,44.78,1.3950609165548327e+45,45.14459317187199,7.925013304804734e+44,3.801893963205597e+43,6.025595860743593e+44,7.925013304804734e+44,0.0,6.025595860743593e+44
217.70446,32.30566,19.34,0.92,-99.99,11/0,0.19,117.86,-0.28,-0.986,0.0,0.15,129.12,-0.636,-0.513,0.0,0.0,0.74,1.0,82.43,k/m,42.943999999999996,-0.13076828026902382,-28.27,5.0,0.5854800936768151,42.594,43.3,43.58,2.3879072503684677e+43,43.37801745418454,3.9264493539960094e+42,1.9952623149688666e+43,3.801893963205597e+43,3.9264493539960094e+42,1.9952623149688666e+43,0.0
217.48821,32.30604,19.71,0.03,-99.99,9/1,0.41,1.13,0.0,-1.56,0.629,0.37,0.63,-0.693,0.0,0.0,0.51,2.96,0.0,29.92,f/g,42.887,0.4712917110589386,-28.0,6.0,0.7025761124121781,42.019999999999996,43.58,44.208999999999996,1.6285513231235601e+44,44.211801449728114,1.04712854805089e+42,3.801893963205597e+43,1.6180800376430512e+44,1.04712854805089e+42,0.0,1.6180800376430512e+44
217.36268,32.3061,18.71,0.03,-99.99,12/0,0.21,12.43,0.0,-0.735,0.325,0.21,13.6,-0.711,0.0,0.0,0.326,0.7,0.0,57.12,f/g,42.869,-0.1549019599857432,-27.95,7.0,0.8196721311475411,42.845,43.58,43.905,8.735103217858466e+43,43.94126804116572,6.998419960022716e+42,3.801893963205597e+43,8.035261221856194e+43,6.998419960022716e+42,0.0,8.035261221856194e+43
217.24316,32.30626,22.34,0.95,-99.99,9/2,1.08,2.14,0.0,-1.455,0.627,0.83,1.56,0.239,0.0,-0.597,0.013,2.28,0.03,3.9,f/g,43.818999999999996,0.35793484700045375,-27.94,8.0,0.936768149882904,42.125,43.58,44.207,1.6239808494999064e+44,44.210580903596316,1.333521432163324e+42,3.801893963205597e+43,1.6106456351782731e+44,1.333521432163324e+42,0.0,1.6106456351782731e+44
217.16662,32.30637,21.26,0.13,-99.99,10/1,0.69,3.23,0.0,0.0,0.531,0.62,3.48,-0.314,-1.659,0.0,0.358,0.78,0.0,5.63,f/g,43.266,-0.10790539730951958,-27.79,9.0,1.053864168618267,43.58,43.58,44.111,1.2912192736135256e+44,44.111,3.801893963205597e+43,3.801893963205597e+43,1.2912192736135256e+44,0.0,0.0,1.2912192736135256e+44


In [15]:
lg045 = lg0[lg0['LAGNerg']>=45]

F1045 = lg045[lg045['F']>=10]
F545 = lg045[lg045['F']>=5]
F2045 = lg045[lg045['F']>=20]

In [16]:
from scipy.stats import f

#F = np.linspace(0,50,101)
F = [5,10,20]  # use specfic values to print the value below 

N = []
for i in F:
    N.append(len(l0[l0['F']>= i]))


N45 = []
for i in F:
    N45.append(len(lg045[lg045['F']>= i]))

    
#dfn, dfd = 2, 9
#vals = f.ppf([0.001, 0.995, 0.999], dfn, dfd)
#print (vals)    
    
#defining degrees of freedom in f-test
dfn, dfd = 2, 9
Prob = f.cdf(F, dfn, dfd)

P = np.subtract(1,Prob)
spurious = np.multiply(N,P)
nospurious = np.subtract(N,spurious)
spuriousfrac = np.divide(spurious, N)

print ("F-value =", F)
print ("Probality =", P)
print ("Number in sample with no L =", N)
print ("Number of spurious in sample =", spurious)
print ("Fraction of spurious in sample =", spuriousfrac)
#print ("Number of none spurious in sample =", nospurious)

F-value = [5, 10, 20]
Probality = [0.03464975 0.00516774 0.00048776]
Number in sample with no L = [41476, 16907, 5760]
Number of spurious in sample = [1437.13297372   87.3709091     2.80951539]
Fraction of spurious in sample = [0.03464975 0.00516774 0.00048776]


In [17]:
new_table = Table([F, P, N45, spurious, spuriousfrac, nospurious])
from astropy.io import ascii
ascii.write(new_table, 'new_table.fits', names=['F', 'P', 'N45', 'spurious', 'spuriousfrac', 'nospurious'], overwrite=True)

new_table.rename_column('col0', 'F')  
new_table.rename_column('col1', 'P') 
new_table.rename_column('col2', 'N45')  
new_table.rename_column('col3', 'spurious') 
new_table.rename_column('col4', 'spuriousfrac')  
new_table.rename_column('col5', 'nospurious')  

#new_table.show_in_browser(jsviewer=True)  

#note that perhaps spuriousfrac should actually be fraction of the AGN identified not fraction of the total in sample

In [18]:
l0Sbc = l0[l0['LSbc']!=0.0]
l0Ell = l0[l0['LEll']!=0.0]
l0Irr = l0[l0['LIrr']!=0.0]

LSbcerg = MaskedColumn(name = "LSbcerg", data = ((l0Sbc['LSbc'])+43.58) , dtype=np.float64) # 44 or 43?
#l0Sbc.add_column(LSbcerg)
l0Sbc.replace_column('LSbcerg',LSbcerg)

LEllerg = MaskedColumn(name = "LEllerg", data = ((l0Ell['LEll'])+43.58) , dtype=np.float64) # 44 or 43?
#l0Ell.add_column(LEllerg)
l0Ell.replace_column('LEllerg',LEllerg)

LIrrerg = MaskedColumn(name = "LIrrerg", data = ((l0Irr['LIrr'])+43.58) , dtype=np.float64) # 44 or 43?
#l0Irr.add_column(LIrrerg)
l0Irr.replace_column('LIrrerg',LIrrerg)



In [19]:
F10Sbc = l0Sbc[l0Sbc['F']>=10]
F5Sbc = l0Sbc[l0Sbc['F']>=5]
F20Sbc = l0Sbc[l0Sbc['F']>=20]

F10Ell = l0Ell[l0Ell['F']>=10]
F5Ell = l0Ell[l0Ell['F']>=5]
F20Ell = l0Ell[l0Ell['F']>=20]

F10Irr = l0Irr[l0Irr['F']>=10]
F5Irr = l0Irr[l0Irr['F']>=5]
F20Irr = l0Irr[l0Irr['F']>=20]


In [20]:
F10 = lg0[lg0['F']>=10]
print (len(F10))
F5 = lg0[lg0['F']>=5]
print (len(F5))
F20 = lg0[lg0['F']>=20]
print (len(F20))

16907
41470
5760


In [21]:
l0Sbcp = l0[l0['LSbcp']!=0.0]
l0Ellp = l0[l0['LELLp']!=0.0]
l0Irrp = l0[l0['LIrrp']!=0.0]

LSbcerg = MaskedColumn(name = "LSbcerg", data = ((l0Sbcp['LSbcp'])+43.58) , dtype=np.float64) # 44 or 43?
#l0Sbcp.add_column(LSbcerg)
l0Sbcp.replace_column('LSbcerg',LSbcerg)

LEllerg = MaskedColumn(name = "LEllerg", data = ((l0Ellp['LELLp'])+43.58) , dtype=np.float64) # 44 or 43?
#l0Ellp.add_column(LEllerg)
l0Ellp.replace_column('LEllerg',LEllerg)

LIrrerg = MaskedColumn(name = "LIrrerg", data = ((l0Irrp['LIrrp'])+43.58) , dtype=np.float64) # 44 or 43?
#l0Irrp.add_column(LIrrerg)
l0Irrp.replace_column('LIrrerg',LIrrerg)


In [22]:
F10Sbcp = l0Sbcp[l0Sbcp['F']>=10]
F5Sbcp = l0Sbcp[l0Sbcp['F']>=5]
F20Sbcp = l0Sbcp[l0Sbcp['F']>=20]

F10Ellp = l0Ellp[l0Ellp['F']>=10]
F5Ellp = l0Ellp[l0Ellp['F']>=5]
F20Ellp = l0Ellp[l0Ellp['F']>=20]

F10Irrp = l0Irrp[l0Irrp['F']>=10]
F5Irrp = l0Irrp[l0Irrp['F']>=5]
F20Irrp = l0Irrp[l0Irrp['F']>=20]


In [23]:
LSbcergp = MaskedColumn(name = "LSbcergp", data = ((l0['LSbcp'])+43.58) , dtype=np.float64) # 44 or 43?
l0.add_column(LSbcergp)
#l0.replace_column('LSbcergp',LSbcergp)

LEllergp = MaskedColumn(name = "LEllergp", data = ((l0['LELLp'])+43.58) , dtype=np.float64) # 44 or 43?
l0.add_column(LEllergp)
#l0.replace_column('LEllergp',LEllergp)

LIrrergp = MaskedColumn(name = "LIrrergp", data = ((l0['LIrrp'])+43.58) , dtype=np.float64) # 44 or 43?
l0.add_column(LIrrergp)
#l0.replace_column('LIrrergp',LIrrergp)



#add columns unlogging 
LSbc1p = MaskedColumn(name = "LSbc1p", data = (10**(l0['LSbcergp'])) , dtype=np.float64) 
l0.add_column(LSbc1p)
#l0.replace_column('LSbc1p',LSbc1p)

LEll1p = MaskedColumn(name = "LEll1p", data = (10**(l0['LEllergp'])) , dtype=np.float64) 
l0.add_column(LEll1p)
#l0.replace_column('LEll1p',LEll1p)

LIrr1p = MaskedColumn(name = "LIrr1p", data = (10**(l0['LIrrergp'])) , dtype=np.float64) 
l0.add_column(LIrr1p)
#l0.replace_column('LIrr1p',LIrr1p)



#thennn add columns replacing 43.58 with 0
LSbc2p = MaskedColumn(name = "LSbc2p", data = (np.where(l0['LSbc1p']==10**43.58,0, l0['LSbc1p'])) , dtype=np.float64) 
l0.add_column(LSbc2p)
#l0.replace_column('LSbc2p',LSbc2p)

LEll2p = MaskedColumn(name = "LEll2p", data = (np.where(l0['LEll1p']==10**43.58,0, l0['LEll1p'])) , dtype=np.float64) 
l0.add_column(LEll2p)
#l0.replace_column('LEll2p',LEll2p)

LIrr2p = MaskedColumn(name = "LIrr2p", data = (np.where(l0['LIrr1p']==10**43.58,0, l0['LIrr1p'])) , dtype=np.float64) 
l0.add_column(LIrr2p)
#l0.replace_column('LIrr2p',LIrr2p)

#then add lg column (sum them)
Lgp = MaskedColumn(name = "Lgp", data = list(map(sum, zip(l0['LEll2p'],l0['LSbc2p'],l0['LIrr2p'])))  , dtype=np.float64)
l0.add_column(Lgp)
#l0.replace_column('Lgp',Lgp )



#then add log column 
LGlogp =  MaskedColumn(name = "LGlogp", data = np.log10(l0['Lgp']) , dtype=np.float64) 
l0.add_column(LGlogp)
#l0.replace_column('LGlogp',LGlogp)

#remove the values where they are all 0
lg0p = l0[[np.isfinite(l0['LGlogp'])]]

F10p = lg0p[lg0p['F']>=10]
F5p = lg0p[lg0p['F']>=5]
F20p = lg0p[lg0p['F']>=20]


print (len(lg0['LAGNerg']))
print (len(F5['LAGNerg']))
print (len(F10['LAGNerg']))
print (len(F20['LAGNerg']))

305254
41470
16907
5760


In [24]:
lg0p.remove_columns(['LIrr2p', 'LEll2p', 'LSbc2p','LIrr1p','LEll1p','LSbc1p','LIrr2','LEll2','LSbc2','LIrr1','LEll1','LSbc1','SA','SType','Chi2S'])

In [25]:
lg0p

RA,Dec,Imag,S,zspec,N,zG,chi2G,LEll,LSbc,LIrr,zGA,chi2GA,LAGNp,LELLp,LSbcp,LIrrp,F,EBV,LAGNerg,logF,I,index,LSbcerg,LEllerg,LIrrerg,Lg,LGlog,LSbcergp,LEllergp,LIrrergp,Lgp,LGlogp
float64,float64,float64,float64,float64,str4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
217.18223,32.30523,22.77,0.95,-99.99,8/3,1.27,2.53,0.0,0.829,0.361,1.2,2.9,0.651,0.0,0.768,0.0,0.54,0.1,44.231,-0.26760624017703144,-30.18,1.0,44.409,43.58,43.940999999999995,3.437455404905813e+44,44.53623707264808,44.348,43.58,43.58,2.2284351492702984e+44,44.348
217.86705,32.30527,20.02,0.03,-99.99,12/0,0.54,37.08,0.44,0.47,-0.461,0.51,45.75,0.268,0.546,0.0,-0.237,0.24,0.5,43.848,-0.619788758288394,-29.15,2.0,44.05,44.019999999999996,43.119,2.3006694855720697e+44,44.36185423235735,43.58,44.126,43.342999999999996,1.5568881628517787e+44,44.19225741667808
217.88212,32.30531,22.67,0.03,-99.99,11/1,0.85,0.55,0.0,0.0,0.121,0.81,0.63,-0.31,0.0,0.0,-0.052,0.53,0.04,43.269999999999996,-0.27572413039921095,-28.79,3.0,43.58,43.58,43.701,5.023425895223877e+43,43.701,43.58,43.58,43.528,3.3728730865886787e+43,43.528
217.84705,32.30564,20.44,0.03,-99.99,11/0,1.08,18.79,0.0,1.319,1.2,1.01,23.96,0.802,0.0,1.278,0.947,0.24,0.05,44.382,-0.619788758288394,-28.42,4.0,44.899,43.58,44.78,1.3950609165548327e+45,45.14459317187199,44.858,43.58,44.527,1.0576190485583862e+45,45.024329264200276
217.70446,32.30566,19.34,0.92,-99.99,11/0,0.19,117.86,-0.28,-0.986,0.0,0.15,129.12,-0.636,-0.513,0.0,0.0,0.74,1.0,42.943999999999996,-0.13076828026902382,-28.27,5.0,42.594,43.3,43.58,2.3879072503684677e+43,43.37801745418454,43.58,43.067,43.58,1.166809617060963e+43,43.067
217.48821,32.30604,19.71,0.03,-99.99,9/1,0.41,1.13,0.0,-1.56,0.629,0.37,0.63,-0.693,0.0,0.0,0.51,2.96,0.0,42.887,0.4712917110589386,-28.0,6.0,42.019999999999996,43.58,44.208999999999996,1.6285513231235601e+44,44.211801449728114,43.58,43.58,44.089999999999996,1.2302687708123711e+44,44.089999999999996
217.36268,32.3061,18.71,0.03,-99.99,12/0,0.21,12.43,0.0,-0.735,0.325,0.21,13.6,-0.711,0.0,0.0,0.326,0.7,0.0,42.869,-0.1549019599857432,-27.95,7.0,42.845,43.58,43.905,8.735103217858466e+43,43.94126804116572,43.58,43.58,43.906,8.053784411990643e+43,43.906
217.24316,32.30626,22.34,0.95,-99.99,9/2,1.08,2.14,0.0,-1.455,0.627,0.83,1.56,0.239,0.0,-0.597,0.013,2.28,0.03,43.818999999999996,0.35793484700045375,-27.94,8.0,42.125,43.58,44.207,1.6239808494999064e+44,44.210580903596316,42.983,43.58,43.592999999999996,4.879031049461457e+43,43.68833358190953
217.16662,32.30637,21.26,0.13,-99.99,10/1,0.69,3.23,0.0,0.0,0.531,0.62,3.48,-0.314,-1.659,0.0,0.358,0.78,0.0,43.266,-0.10790539730951958,-27.79,9.0,43.58,43.58,44.111,1.2912192736135256e+44,44.111,43.58,41.921,43.937999999999995,8.752986876044035e+43,43.942156277300356


In [26]:
LG045 = lg0p[lg0p['LAGNerg']>=45]
L045 = l0[l0['LAGNerg']>=45]

In [27]:
LG0cut1045 = LG045[LG045['F']>=10]
L0cut1045 = L045[L045['F']>=10]

In [28]:
L0cut45LESSTHAN10 = L045[L045['F']<=10]

In [29]:
LG0cut10 = lg0p[lg0p['F']>=10]
L0cut10 = l0[l0['F']>=10]

In [30]:
L0cut45LESSTHAN10

RA,Dec,Imag,S,zspec,N,zG,chi2G,LEll,LSbc,LIrr,zGA,chi2GA,LAGNp,LELLp,LSbcp,LIrrp,F,EBV,Chi2S,SType,LAGNerg,logF,I,index,SA,LSbcerg,LEllerg,LIrrerg,Lg,LGlog,LSbc1,LEll1,LIrr1,LSbc2,LEll2,LIrr2,LSbcergp,LEllergp,LIrrergp,LSbc1p,LEll1p,LIrr1p,LSbc2p,LEll2p,LIrr2p,Lgp,LGlogp
float64,float64,float64,float64,float64,str4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str5,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
217.27482,32.3064,19.08,0.03,-99.99,10/2,0.04,148.17,0.0,-1.412,0.0,2.83,130.97,3.271,1.532,0.0,0.0,1.46,0.1,191.74,g/k,46.851,0.1643528557844371,-27.73,10.0,1.1709601873536302,42.168,43.58,43.58,1.4723125024327162e+42,42.168,1.4723125024327162e+42,3.801893963205597e+43,3.801893963205597e+43,1.4723125024327162e+42,0.0,0.0,43.58,45.111999999999995,43.58,3.801893963205597e+43,1.2941958414499704e+45,3.801893963205597e+43,0.0,1.2941958414499704e+45,0.0,1.2941958414499704e+45,45.111999999999995
217.82347,32.30846,18.5,0.03,-99.99,11/0,2.99,256.33,2.252,0.0,2.111,3.0,303.93,2.353,2.328,0.0,0.0,0.45,0.05,220.25,k/m,45.933,-0.3467874862246563,-25.59,42.0,4.918032786885246,43.58,45.832,45.690999999999995,1.1701115087697834e+46,46.068227250922064,3.801893963205597e+43,6.792036326171856e+45,4.9090787615259786e+45,0.0,6.792036326171856e+45,4.9090787615259786e+45,43.58,45.908,43.58,3.801893963205597e+43,8.090958991783846e+45,3.801893963205597e+43,0.0,8.090958991783846e+45,0.0,8.090958991783846e+45,45.908
217.43907,32.30938,17.92,0.85,-99.99,9/1,1.91,217.32,0.0,2.199,1.205,1.84,321.62,1.719,0.0,2.136,0.0,0.03,0.13,212.97,K/M,45.299,-1.5228787452803376,-24.03,61.0,7.142857142857143,45.778999999999996,43.58,44.785,6.621274272072895e+45,45.82094157797054,6.01173737483273e+45,3.801893963205597e+43,6.095368972401644e+44,6.01173737483273e+45,0.0,6.095368972401644e+44,45.716,43.58,43.58,5.199959965335173e+45,3.801893963205597e+43,3.801893963205597e+43,5.199959965335173e+45,0.0,0.0,5.199959965335173e+45,45.716
217.227,32.31004,22.28,0.04,-99.99,8/3,0.1,4.6,0.0,0.0,-1.713,2.11,3.05,1.621,-0.346,0.0,0.0,2.78,0.04,4.15,a/f,45.201,0.4440447959180762,15.89,76.0,8.899297423887589,43.58,43.58,41.867,7.362070974947316e+41,41.867,3.801893963205597e+43,3.801893963205597e+43,7.362070974947316e+41,0.0,0.0,7.362070974947316e+41,43.58,43.234,43.58,3.801893963205597e+43,1.7139573075084322e+43,3.801893963205597e+43,0.0,1.7139573075084322e+43,0.0,1.7139573075084322e+43,43.234
217.9176,32.31062,22.57,0.67,-99.99,9/3,1.33,1.34,0.0,0.0,0.926,1.89,1.08,1.484,0.0,0.446,0.0,1.85,0.06,2.41,a/f,45.064,0.26717172840301384,16.23,105.0,12.295081967213116,43.58,43.58,44.506,3.2062693245054673e+44,44.506,3.801893963205597e+43,3.801893963205597e+43,3.2062693245054673e+44,0.0,0.0,3.2062693245054673e+44,44.025999999999996,43.58,43.58,1.0616955571987154e+44,3.801893963205597e+43,3.801893963205597e+43,1.0616955571987154e+44,0.0,0.0,1.0616955571987154e+44,44.025999999999996
216.20163,32.31104,20.98,0.76,-99.99,10/0,0.45,9.66,0.0,-0.214,-1.218,3.0,9.57,2.255,0.516,0.0,0.0,1.03,0.1,4.15,k/m,45.835,0.012837224705172217,16.31,119.0,13.934426229508198,43.366,43.58,42.361999999999995,2.552880978097953e+43,43.40703056734626,2.322736796357105e+43,3.801893963205597e+43,2.3014418174084808e+42,2.322736796357105e+43,0.0,2.3014418174084808e+42,43.58,44.096,43.58,3.801893963205597e+43,1.247383514242933e+44,3.801893963205597e+43,0.0,1.247383514242933e+44,0.0,1.247383514242933e+44,44.096
217.36603,32.31109,19.35,0.03,-99.99,10/3,2.99,113.91,2.206,0.0,2.148,3.0,130.12,2.507,2.247,0.0,0.0,0.44,0.06,138.26,g/k,46.086999999999996,-0.3565473235138126,16.31,121.0,14.168618266978925,43.58,45.786,45.728,1.1455063843025474e+46,46.05899751400524,3.801893963205597e+43,6.10942024905574e+45,5.3456435939697356e+45,0.0,6.10942024905574e+45,5.3456435939697356e+45,43.58,45.827,4

In [31]:
L0cut10

RA,Dec,Imag,S,zspec,N,zG,chi2G,LEll,LSbc,LIrr,zGA,chi2GA,LAGNp,LELLp,LSbcp,LIrrp,F,EBV,Chi2S,SType,LAGNerg,logF,I,index,SA,LSbcerg,LEllerg,LIrrerg,Lg,LGlog,LSbc1,LEll1,LIrr1,LSbc2,LEll2,LIrr2,LSbcergp,LEllergp,LIrrergp,LSbc1p,LEll1p,LIrr1p,LSbc2p,LEll2p,LIrr2p,Lgp,LGlogp
float64,float64,float64,float64,float64,str4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str5,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
217.44642,32.30949,22.64,0.33,-99.99,10/2,1.5,3.18,0.0,0.0,1.229,1.1,0.53,0.815,0.0,0.105,-0.176,20.86,0.0,5.03,b/a,44.394999999999996,1.319314304090512,-23.02,64.0,7.494145199063232,43.58,43.58,44.809,6.441692655151736e+44,44.809,3.801893963205597e+43,3.801893963205597e+43,6.441692655151736e+44,0.0,0.0,6.441692655151736e+44,43.684999999999995,43.58,43.403999999999996,4.84172367584094e+43,3.801893963205597e+43,2.535128630497886e+43,4.84172367584094e+43,0.0,2.535128630497886e+43,7.376852306338826e+43,43.86787108843545
217.46803,32.31347,22.24,0.04,-99.99,10/3,0.91,3.99,0.0,0.0,0.513,1.91,1.33,1.15,0.0,0.0,-0.816,10.0,0.0,2.36,O/B,44.73,1.0,16.64,224.0,26.229508196721316,43.58,43.58,44.092999999999996,1.238796586530359e+44,44.092999999999996,3.801893963205597e+43,3.801893963205597e+43,1.238796586530359e+44,0.0,0.0,1.238796586530359e+44,43.58,43.58,42.763999999999996,3.801893963205597e+43,3.801893963205597e+43,5.807644175213064e+42,0.0,0.0,5.807644175213064e+42,5.807644175213064e+42,42.763999999999996
216.23924,32.31401,23.56,0.22,-99.99,9/3,3.0,70.22,2.445,0.0,0.0,0.83,7.08,2.098,0.0,0.0,-0.656,36.66,1.58,146.43,1500K,45.678,1.5641924606261979,16.69,246.0,28.8056206088993,43.58,46.025,43.58,1.0592537251772854e+46,46.025,3.801893963205597e+43,1.0592537251772854e+46,3.801893963205597e+43,0.0,1.0592537251772854e+46,0.0,43.58,43.58,42.924,3.801893963205597e+43,3.801893963205597e+43,8.394599865193965e+42,0.0,0.0,8.394599865193965e+42,8.394599865193965e+42,42.924
217.19404,32.31454,22.73,0.51,-99.99,9/4,0.07,2.14,0.0,0.0,-2.212,1.9,0.64,1.199,0.0,-0.414,0.0,11.45,0.02,1.33,a/f,44.778999999999996,1.0588054866759067,16.74,269.0,31.498829039812648,43.58,43.58,41.367999999999995,2.3334580622809758e+41,41.367999999999995,3.801893963205597e+43,3.801893963205597e+43,2.3334580622809758e+41,0.0,0.0,2.3334580622809758e+41,43.166,43.58,43.58,1.4655478409559007e+43,3.801893963205597e+43,3.801893963205597e+43,1.4655478409559007e+43,0.0,0.0,1.4655478409559007e+43,43.166
217.76649,32.31627,21.26,0.03,-99.99,11/1,1.28,4.56,0.0,0.0,1.316,0.89,1.22,0.535,-0.041,0.0,0.7,10.61,0.0,21.73,f/g,44.114999999999995,1.0257153839013406,16.89,351.0,41.10070257611242,43.58,43.58,44.896,7.870457896951001e+44,44.896,3.801893963205597e+43,3.801893963205597e+43,7.870457896951001e+44,0.0,0.0,7.870457896951001e+44,43.58,43.539,44.28,3.801893963205597e+43,3.459393778261231e+43,1.9054607179632523e+44,0.0,3.459393778261231e+43,1.9054607179632523e+44,2.2514000957893754e+44,44.35245268023076
217.32876,32.31811,-26.1,0.68,-99.99,11/3,2.39,9.05,0.0,1.7,0.0,1.84,2.68,2.187,0.0,0.534,-1.324,12.88,0.79,12.79,1500K,45.766999999999996,1.1099158630237933,17.0,433.0,50.70257611241218,45.28,43.58,43.58,1.9054607179632523e+45,45.28,1.9054607179632523e+45,3.801893963205597e+43,3.801893963205597e+43,1.9054607179632523e+45,0.0,0.0,44.114,43.58,42.256,1.300169578033282e+44,3.801893963205597e+43,1.8030177408595698e+42,1.300169578033282e+44,0.0,1.8030177408595698e+42,1.3181997554418778e+44,44.11998122673197
216.77754,32.31943,24.08,0.11,-99.99,12/1,1.89,12.53,0.0,1.549,0.0,2.26,2.85,2.485,0.0,1.241,-0.04,16.32,0.5,15.4,1500K,46.065,1.2127201544178423,17.09,492.0,57.6112412177986,45.129,43.58,43.58,1.3458603540559414e+45,45.129,1.3458603540559414e+45,3.801893963205597e+43,3.801893963205597e+43,1.3458603540559414e+45,0.0,0.0,44.821,43.58,43.54,6.6221650370

In [32]:
l0

RA,Dec,Imag,S,zspec,N,zG,chi2G,LEll,LSbc,LIrr,zGA,chi2GA,LAGNp,LELLp,LSbcp,LIrrp,F,EBV,Chi2S,SType,LAGNerg,logF,I,index,SA,LSbcerg,LEllerg,LIrrerg,Lg,LGlog,LSbc1,LEll1,LIrr1,LSbc2,LEll2,LIrr2,LSbcergp,LEllergp,LIrrergp,LSbc1p,LEll1p,LIrr1p,LSbc2p,LEll2p,LIrr2p,Lgp,LGlogp
float64,float64,float64,float64,float64,str4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str5,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
217.18223,32.30523,22.77,0.95,-99.99,8/3,1.27,2.53,0.0,0.829,0.361,1.2,2.9,0.651,0.0,0.768,0.0,0.54,0.1,8.92,1500K,44.231,-0.26760624017703144,-30.18,1.0,0.117096018735363,44.409,43.58,43.940999999999995,3.437455404905813e+44,44.53623707264808,2.5644840365177108e+44,3.801893963205597e+43,8.729713683881023e+43,2.5644840365177108e+44,0.0,8.729713683881023e+43,44.348,43.58,43.58,2.2284351492702984e+44,3.801893963205597e+43,3.801893963205597e+43,2.2284351492702984e+44,0.0,0.0,2.2284351492702984e+44,44.348
217.86705,32.30527,20.02,0.03,-99.99,12/0,0.54,37.08,0.44,0.47,-0.461,0.51,45.75,0.268,0.546,0.0,-0.237,0.24,0.5,56.73,k/m,43.848,-0.619788758288394,-29.15,2.0,0.234192037470726,44.05,44.019999999999996,43.119,2.3006694855720697e+44,44.36185423235735,1.1220184543019561e+44,1.0471285480508899e+44,1.3152248321922376e+43,1.1220184543019561e+44,1.0471285480508899e+44,1.3152248321922376e+43,43.58,44.126,43.342999999999996,3.801893963205597e+43,1.336595516546435e+44,2.2029264630534381e+43,0.0,1.336595516546435e+44,2.2029264630534381e+43,1.5568881628517787e+44,44.19225741667808
217.88212,32.30531,22.67,0.03,-99.99,11/1,0.85,0.55,0.0,0.0,0.121,0.81,0.63,-0.31,0.0,0.0,-0.052,0.53,0.04,1.09,f/g,43.269999999999996,-0.27572413039921095,-28.79,3.0,0.35128805620608905,43.58,43.58,43.701,5.023425895223877e+43,43.701,3.801893963205597e+43,3.801893963205597e+43,5.023425895223877e+43,0.0,0.0,5.023425895223877e+43,43.58,43.58,43.528,3.801893963205597e+43,3.801893963205597e+43,3.3728730865886787e+43,0.0,0.0,3.3728730865886787e+43,3.3728730865886787e+43,43.528
217.84705,32.30564,20.44,0.03,-99.99,11/0,1.08,18.79,0.0,1.319,1.2,1.01,23.96,0.802,0.0,1.278,0.947,0.24,0.05,88.09,f/g,44.382,-0.619788758288394,-28.42,4.0,0.468384074941452,44.899,43.58,44.78,1.3950609165548327e+45,45.14459317187199,7.925013304804734e+44,3.801893963205597e+43,6.025595860743593e+44,7.925013304804734e+44,0.0,6.025595860743593e+44,44.858,43.58,44.527,7.211074791828946e+44,3.801893963205597e+43,3.3651156937549153e+44,7.211074791828946e+44,0.0,3.3651156937549153e+44,1.0576190485583862e+45,45.024329264200276
217.70446,32.30566,19.34,0.92,-99.99,11/0,0.19,117.86,-0.28,-0.986,0.0,0.15,129.12,-0.636,-0.513,0.0,0.0,0.74,1.0,82.43,k/m,42.943999999999996,-0.13076828026902382,-28.27,5.0,0.5854800936768151,42.594,43.3,43.58,2.3879072503684677e+43,43.37801745418454,3.9264493539960094e+42,1.9952623149688666e+43,3.801893963205597e+43,3.9264493539960094e+42,1.9952623149688666e+43,0.0,43.58,43.067,43.58,3.801893963205597e+43,1.166809617060963e+43,3.801893963205597e+43,0.0,1.166809617060963e+43,0.0,1.166809617060963e+43,43.067
217.48821,32.30604,19.71,0.03,-99.99,9/1,0.41,1.13,0.0,-1.56,0.629,0.37,0.63,-0.693,0.0,0.0,0.51,2.96,0.0,29.92,f/g,42.887,0.4712917110589386,-28.0,6.0,0.7025761124121781,42.019999999999996,43.58,44.208999999999996,1.6285513231235601e+44,44.211801449728114,1.04712854805089e+42,3.801893963205597e+43,1.6180800376430512e+44,1.04712854805089e+42,0.0,1.6180800376430512e+44,43.58,43.58,44.089999999999996,3.801893963205597e+43,3.801893963205597e+43,1.2302687708123711e+44,0.0,0.0,1.2302687708123711e+44,1.2302687708123711e+44,44.089999999999996
217.36268,32.3061,18.71,0.03,-99.99,12/0,0.21,12.43,0.0,-0.735,0.325,0.21,13.6,-0.711,0.0,0.0,0.326,0.7,0.0,57.12,f/g,42.869,-0.1549019599857432,-27.95,7.0,0.8196721311475411,42.845,43.58,43.905,8.735103

In [33]:
from astropy.io import ascii
ascii.write(LG045, 'LG045.fits', overwrite=True, comment=False)

In [34]:
from astropy.io import ascii
ascii.write(l0, 'l0.fits', overwrite=True, comment = False, format = 'no_header')
#agn luminosity is 0 removed

In [35]:
from astropy.io import ascii
ascii.write(lg0p, 'lg0p.fits', overwrite=True, comment = False, format = 'no_header')
#galaxy luminosity (agn model) is 0 removed

In [36]:
from astropy.io import ascii
ascii.write(LG0cut1045, 'LG0cut1045.fits', overwrite=True, comment = False, format = 'no_header')
#galaxy luminosity (agn model) is 0 removed and cut 10 and 45

In [37]:
from astropy.io import ascii
ascii.write(L0cut1045, 'L0cut1045.fits', overwrite=True, comment = False, format = 'no_header')
#agn luminosity is 0 removed and cut 10 and 45

In [38]:
from astropy.io import ascii
ascii.write(LG0cut10, 'LG0cut10.fits', overwrite=True, comment = False, format = 'no_header')
#galaxy luminosity (agn model) is 0 removed and cut 10 

In [39]:
from astropy.io import ascii
ascii.write(L0cut10, 'L0cut10.fits', overwrite=True, comment = False, format = 'no_header')
#agn luminosity is 0 removed and cut 10 

In [40]:
# so LGlogp is all the galaxy luminositys from the AGN model added up with row removed if all are 0 and any 0 values other than that not counted in sums. 

In [41]:
from astropy.io import ascii
ascii.write(LG0cut10, 'LG0cut10SF.csv', overwrite=True, comment = False)
#galaxy luminosity (agn model) is 0 removed and cut 10 

In [42]:
from astropy.io import ascii
ascii.write(LG0cut1045, 'LG0cut1045.csv', overwrite=True, comment = False)
#galaxy luminosity (agn model) is 0 removed and cut 10 and 45

In [43]:
AV = [i*3.2 for i in L0cut10['EBV']]

AV = MaskedColumn(name = "AV", data = AV  , dtype=np.float64) 
L0cut10.add_column(AV)
#L0cut10.replace_column('AV',AV)


In [44]:
T1L0cut10 = L0cut10[L0cut10['AV']<1]
T2L0cut10 = L0cut10[L0cut10['AV']>1]

In [45]:
from astropy.io import ascii
ascii.write(T1L0cut10, 'T1L0cut10.fits', overwrite=True, comment = False, format = 'no_header')
#agn luminosity is 0 removed and cut 10 for T1

In [46]:
from astropy.io import ascii
ascii.write(T2L0cut10, 'T2L0cut10.fits', overwrite=True, comment = False, format = 'no_header')
#agn luminosity is 0 removed and cut 10 for T2

In [47]:
AV = [i*3.2 for i in L0cut1045['EBV']]

AV = MaskedColumn(name = "AV", data = AV  , dtype=np.float64) 
L0cut1045.add_column(AV)
#L0cut1045.replace_column('AV',AV)


In [48]:
T1L0cut1045 = L0cut1045[L0cut1045['AV']<1]
T2L0cut1045 = L0cut1045[L0cut1045['AV']>1]

In [49]:
from astropy.io import ascii
ascii.write(T1L0cut1045, 'T1L0cut1045.fits', overwrite=True, comment = False)
#agn luminosity is 0 removed and cut 1045 for T1

In [50]:
from astropy.io import ascii
ascii.write(T2L0cut1045, 'T2L0cut1045.fits', overwrite=True, comment = False)
#agn luminosity is 0 removed and cut 1045 for T2

In [51]:
np.array([[1,2,3],[4,5,6]])

array([[1, 2, 3],
       [4, 5, 6]])

In [52]:
[216.23924,217.32876,216.77754,217.40246,217.17911,217.42245,217.97138,217.78001,217.33069,217.39288]

[216.23924,
 217.32876,
 216.77754,
 217.40246,
 217.17911,
 217.42245,
 217.97138,
 217.78001,
 217.33069,
 217.39288]

In [53]:
32.31401,32.31811,32.31943,32.32228,32.32282,32.34531,32.36198,32.36476,32.36947,32.37035

(32.31401,
 32.31811,
 32.31943,
 32.32228,
 32.32282,
 32.34531,
 32.36198,
 32.36476,
 32.36947,
 32.37035)

In [54]:
T2L0cut1045

RA,Dec,Imag,S,zspec,N,zG,chi2G,LEll,LSbc,LIrr,zGA,chi2GA,LAGNp,LELLp,LSbcp,LIrrp,F,EBV,Chi2S,SType,LAGNerg,logF,I,index,SA,LSbcerg,LEllerg,LIrrerg,Lg,LGlog,LSbc1,LEll1,LIrr1,LSbc2,LEll2,LIrr2,LSbcergp,LEllergp,LIrrergp,LSbc1p,LEll1p,LIrr1p,LSbc2p,LEll2p,LIrr2p,Lgp,LGlogp,AV
float64,float64,float64,float64,float64,str4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str5,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
216.23924,32.31401,23.56,0.22,-99.99,9/3,3.0,70.22,2.445,0.0,0.0,0.83,7.08,2.098,0.0,0.0,-0.656,36.66,1.58,146.43,1500K,45.678,1.5641924606261979,16.69,246.0,28.8056206088993,43.58,46.025,43.58,1.0592537251772854e+46,46.025,3.801893963205597e+43,1.0592537251772854e+46,3.801893963205597e+43,0.0,1.0592537251772854e+46,0.0,43.58,43.58,42.924,3.801893963205597e+43,3.801893963205597e+43,8.394599865193965e+42,0.0,0.0,8.394599865193965e+42,8.394599865193965e+42,42.924,5.056000000000001
217.32876,32.31811,-26.1,0.68,-99.99,11/3,2.39,9.05,0.0,1.7,0.0,1.84,2.68,2.187,0.0,0.534,-1.324,12.88,0.79,12.79,1500K,45.766999999999996,1.1099158630237933,17.0,433.0,50.70257611241218,45.28,43.58,43.58,1.9054607179632523e+45,45.28,1.9054607179632523e+45,3.801893963205597e+43,3.801893963205597e+43,1.9054607179632523e+45,0.0,0.0,44.114,43.58,42.256,1.300169578033282e+44,3.801893963205597e+43,1.8030177408595698e+42,1.300169578033282e+44,0.0,1.8030177408595698e+42,1.3181997554418778e+44,44.11998122673197,2.5280000000000005
216.77754,32.31943,24.08,0.11,-99.99,12/1,1.89,12.53,0.0,1.549,0.0,2.26,2.85,2.485,0.0,1.241,-0.04,16.32,0.5,15.4,1500K,46.065,1.2127201544178423,17.09,492.0,57.6112412177986,45.129,43.58,43.58,1.3458603540559414e+45,45.129,1.3458603540559414e+45,3.801893963205597e+43,3.801893963205597e+43,1.3458603540559414e+45,0.0,0.0,44.821,43.58,43.54,6.6221650370175885e+44,3.801893963205597e+43,3.4673685045253096e+43,6.6221650370175885e+44,0.0,3.4673685045253096e+43,6.9689018874701195e+44,44.84316435029544,1.6
217.40246,32.32228,22.19,0.22,-99.99,13/1,2.44,5.52,0.0,2.059,1.345,1.34,0.7,1.733,0.998,0.754,0.703,32.12,0.5,25.39,1500K,45.312999999999995,1.5067755366066433,17.22,606.0,70.96018735362999,45.638999999999996,43.58,44.925,5.196513878500833e+45,45.71571209150841,4.3551187368556436e+45,3.801893963205597e+43,8.413951416451896e+44,4.3551187368556436e+45,0.0,8.413951416451896e+44,44.333999999999996,44.577999999999996,44.283,2.157744409152647e+44,3.784425847170897e+44,1.9186687406702948e+44,2.157744409152647e+44,3.784425847170897e+44,1.9186687406702948e+44,7.860838996993839e+44,44.8954689012963,1.6
217.17911,32.32282,21.19,0.07,-99.99,13/0,2.65,86.06,0.0,2.037,2.073,1.21,11.55,2.578,0.0,0.8,1.223,30.02,3.98,184.11,f/g,46.158,1.4774106879072515,17.25,625.0,73.18501170960188,45.617,43.58,45.653,8.637795297130144e+45,45.93640290770173,4.1399967481972786e+45,3.801893963205597e+43,4.497798548932866e+45,4.1399967481972786e+45,0.0,4.497798548932866e+45,44.379999999999995,43.58,44.803,2.3988329190194653e+44,3.801893963205597e+43,6.353309318517397e+44,2.3988329190194653e+44,0.0,6.353309318517397e+44,8.752142237536861e+44,44.942114367087555,12.736
217.42245,32.34531,22.4,0.05,-99.99,14/0,1.36,2.35,0.0,1.467,-0.474,1.18,0.54,1.45,0.915,0.343,0.061,17.92,0.5,11.32,1500K,45.03,1.2533380053261065,17.86,1619.0,189.57845433255272,45.047,43.58,43.106,1.1270589216798358e+45,45.05194662118498,1.1142945335917224e+45,3.801893963205597e+43,1.2764388088113487e+43,1.1142945335917224e+45,0.0,1.2764388088113487e+43,43.923,44.495,43.641,8.375292821268861e+43,3.1260793671239365e+44,4.375221051582503e+43,8.375292821268861e+43,3.1260793671239365e+44,4.375221051582503e+43,4.401130754409073e+44,44.64356427132932,1.6
217.97138,32.36198,22.14,0.03,-99.99,14/0,1.78,8.17,0.0,1.9,0.0,1.33,1.85,1.875,0.375,1.365,0.0,16.37,0.63,34.28,1

In [55]:
from astropy.io import ascii
ascii.write(L0cut45LESSTHAN10, 'L0cut45LESSTHAN10.fits', overwrite=True, comment = False)
#agn luminosity is 0 removed and cut 10 and 45

In [56]:
LG0cut1045

RA,Dec,Imag,S,zspec,N,zG,chi2G,LEll,LSbc,LIrr,zGA,chi2GA,LAGNp,LELLp,LSbcp,LIrrp,F,EBV,LAGNerg,logF,I,index,LSbcerg,LEllerg,LIrrerg,Lg,LGlog,LSbcergp,LEllergp,LIrrergp,Lgp,LGlogp
float64,float64,float64,float64,float64,str4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
216.23924,32.31401,23.56,0.22,-99.99,9/3,3.0,70.22,2.445,0.0,0.0,0.83,7.08,2.098,0.0,0.0,-0.656,36.66,1.58,45.678,1.5641924606261979,16.69,246.0,43.58,46.025,43.58,1.0592537251772854e+46,46.025,43.58,43.58,42.924,8.394599865193965e+42,42.924
217.32876,32.31811,-26.1,0.68,-99.99,11/3,2.39,9.05,0.0,1.7,0.0,1.84,2.68,2.187,0.0,0.534,-1.324,12.88,0.79,45.766999999999996,1.1099158630237933,17.0,433.0,45.28,43.58,43.58,1.9054607179632523e+45,45.28,44.114,43.58,42.256,1.3181997554418778e+44,44.11998122673197
216.77754,32.31943,24.08,0.11,-99.99,12/1,1.89,12.53,0.0,1.549,0.0,2.26,2.85,2.485,0.0,1.241,-0.04,16.32,0.5,46.065,1.2127201544178423,17.09,492.0,45.129,43.58,43.58,1.3458603540559414e+45,45.129,44.821,43.58,43.54,6.9689018874701195e+44,44.84316435029544
217.95706,32.3197,21.65,0.98,-99.99,11/3,0.01,13.61,0.0,0.0,-3.513,1.93,2.61,1.753,0.69,0.0,0.0,22.13,0.03,45.333,1.344981413927258,17.11,505.0,43.58,43.58,40.067,1.166809617060963e+40,40.067,43.58,44.269999999999996,43.58,1.8620871366628504e+44,44.269999999999996
217.40246,32.32228,22.19,0.22,-99.99,13/1,2.44,5.52,0.0,2.059,1.345,1.34,0.7,1.733,0.998,0.754,0.703,32.12,0.5,45.312999999999995,1.5067755366066433,17.22,606.0,45.638999999999996,43.58,44.925,5.196513878500833e+45,45.71571209150841,44.333999999999996,44.577999999999996,44.283,7.860838996993839e+44,44.8954689012963
217.17911,32.32282,21.19,0.07,-99.99,13/0,2.65,86.06,0.0,2.037,2.073,1.21,11.55,2.578,0.0,0.8,1.223,30.02,3.98,46.158,1.4774106879072515,17.25,625.0,45.617,43.58,45.653,8.637795297130144e+45,45.93640290770173,44.379999999999995,43.58,44.803,8.752142237536861e+44,44.942114367087555
217.95201,32.3288,21.91,0.03,-99.99,11/3,0.02,7.11,0.0,0.0,-3.051,1.74,1.19,1.657,0.0,0.071,0.0,23.4,0.06,45.236999999999995,1.3692158574101427,17.48,900.0,43.58,43.58,40.528999999999996,3.3806483620597876e+40,40.528999999999996,43.650999999999996,43.58,43.58,4.477133041763587e+43,43.650999999999996
216.52493,32.33303,21.94,0.85,-99.99,12/0,0.02,6.74,0.0,0.0,-2.912,2.66,1.6,1.89,0.0,0.0,-0.012,12.25,0.0,45.47,1.0881360887005513,17.59,1082.0,43.58,43.58,40.668,4.655860935229582e+40,40.668,43.58,43.58,43.568,3.6982817978026436e+43,43.568
216.43354,32.3436,20.17,0.07,-99.99,12/0,1.87,33.79,0.0,2.327,1.92,0.88,7.77,1.951,1.0,0.0,-0.077,14.4,0.25,45.531,1.1583624920952498,17.83,1556.0,45.907,43.58,45.5,1.1234627962656696e+46,46.05055869522951,43.58,44.58,43.503,4.12031371537821e+44,44.61493028391439


In [57]:
from astropy.io import ascii
ascii.write(LG0cut1045, 'LG0cut1045.csv', overwrite=True, comment = False)